## 步骤 1 - 安装所需的 Python 环境及包

In [1]:
!pip install pandas
!pip install zeno_client

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ------- -------------------------------- 2.1/11.3 MB 14.7 MB/s eta 0:00:01
   ------------------------ --------------- 7.1/11.3 MB 19.9 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.3 MB 20.7 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 18.7 MB/s  0:00:00
   ---------------------------------------- 0.0/13.1 MB ? eta -:--:--
   -------------- ------------------------- 4.7/13.1 MB 23.8 MB/s eta 0:00:01
   ---------------------------- ----------- 9.4/13.1 MB 23.5 MB/s eta 0:00:01
   ---------------------------------------- 13.1/13.1 MB 21.6 MB/s  0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/4 [pytz]
   -------------

## 步骤 2 - 初步读入并清洗数据

In [4]:
import pandas as pd

df = pd.read_csv("./tweets.csv")
df.head(5)

,text,label,roberta,roberta_score,gpt2,gpt2_score
0,@user @user what do these '1/2 naked pics' hav...,neutral,0,0.8047260642051697,LABEL_2,0.9134505987167358
1,OH: âI had a blue penis while I was thisâ?[...,neutral,1,0.8669487237930298,LABEL_1,0.7534046173095703
2,"@user @user That's coming, but I think the vic...",neutral,1,0.7637239098548889,LABEL_2,0.9999619722366333
3,I think I may be finally in with the in crowd ...,positive,2,0.7740470767021179,LABEL_2,0.8987836837768555
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",negative,1,0.41639748215675354,LABEL_2,0.9864314198493958


我们可以发现，Roberta 使用 0/1/2 来分别代表 negative/neutral/positive，而 gpt2 使用 LABEL_0/1/2 来代表 negative/neutral/positive。为了方便后续处理，我们需要进行清理。

In [12]:
labels = ["negative","neutral","positive"]
def label_map_gpt2(x):
    try:
        y = labels[int(x.split("_")[1])]
    except:
        y = x
    return y
    
def label_map_roberta(x):
    try:
        y = labels[int(x)]
    except:
        y = x
    return y

df['gpt2'] = df['gpt2'].map(label_map_gpt2)
df['roberta'] = df['roberta'].map(label_map_roberta)
df.head(5)

,text,label,roberta,roberta_score,gpt2,gpt2_score
0,@user @user what do these '1/2 naked pics' hav...,neutral,negative,0.8047260642051697,positive,0.9134505987167358
1,OH: âI had a blue penis while I was thisâ?[...,neutral,neutral,0.8669487237930298,neutral,0.7534046173095703
2,"@user @user That's coming, but I think the vic...",neutral,neutral,0.7637239098548889,positive,0.9999619722366333
3,I think I may be finally in with the in crowd ...,positive,positive,0.7740470767021179,positive,0.8987836837768555
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",negative,neutral,0.41639748215675354,positive,0.9864314198493958


Zeno 处理要求我们增加 input_length 列与 id 列，对数据进行进一步处理

In [13]:
df["input_length"] = df["text"].str.len()
df['id'] = df.index
df.head(5)

,text,label,roberta,roberta_score,gpt2,gpt2_score,input_length,id
0,@user @user what do these '1/2 naked pics' hav...,neutral,negative,0.8047260642051697,positive,0.9134505987167358,96,0
1,OH: âI had a blue penis while I was thisâ?[...,neutral,neutral,0.8669487237930298,neutral,0.7534046173095703,75,1
2,"@user @user That's coming, but I think the vic...",neutral,neutral,0.7637239098548889,positive,0.9999619722366333,87,2
3,I think I may be finally in with the in crowd ...,positive,positive,0.7740470767021179,positive,0.8987836837768555,83,3
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",negative,neutral,0.41639748215675354,positive,0.9864314198493958,133,4


## 步骤 3 - 启动 Zeno 进行模型分析

创建一个 [Zeno](https://hub.zenoml.com/account) 账号，阅读如下代码并正确运行，运行完成后你将在个人账户下看到创建的 projects

In [ ]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd

client = ZenoClient("zen_iqyQKsX2w17SATGGZCAarsfkYw_-Cj3dYhlrNRQNxHY")

# 创建项目
proj = client.create_project(
    name="Twitter Sentiment Analysis",
    view="text-classification",
    metrics=[
        ZenoMetric(name="roberta_accuracy", type="mean", columns=["roberta_correct"]),
        ZenoMetric(name="gpt2_accuracy", type="mean", columns=["gpt2_correct"])
    ]
)

proj.upload_dataset(df, id_column="id", data_column='text', label_column="label")

# 为 Roberta 模型分别创建系统数据框
df_roberta = pd.DataFrame({
    "id": df["id"],
    "output": df["roberta"],
    "roberta_correct": (df["roberta"] == df["label"]).astype(int)
})
proj.upload_system(df_roberta, name="Roberta", id_column="id", output_column="output")

df_gpt2 = pd.DataFrame({
    "id": df["id"],
    "output": df["gpt2"],
    "gpt2_correct": (df["gpt2"] == df["label"]).astype(int)
})
proj.upload_system(df_gpt2, name="GPT2", id_column="id", output_column="output")

c:\Users\leona\anaconda3\envs\isse\Lib\site-packages\zeno_client\client.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
c:\Users\leona\anaconda3\envs\isse\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successfully created project.
Access your project at  https://hub.zenoml.com/project/780fd658-56a9-4e34-bc35-10f94444b8b4/Twitter%20Sentiment%20Analysis


c:\Users\leona\anaconda3\envs\isse\Lib\site-packages\zeno_client\util.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29'
 '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43'
 '44' '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57'
 '58' '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71'
 '72' '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85'
 '86' '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99'
 '100' '101' '102' '103' '104' '105' '106' '107' '108' '109' '110' '111'
 '112' '113' '114' '115' '116' '117' '118' '119' '120' '121' '122' '123'
 '124' '125' '126' '127' '128' '129' '130' '131' '132' '133' '134' '135'
 '136' '137' '138' '139' '140' '141' '142' '143' '144' '145' '146' '147'
 '148' '149' '15

Successfully uploaded data


100%|██████████| 1/1 [00:00<00:00,  1.01it/s]


Successfully uploaded system


100%|██████████| 1/1 [00:00<00:00,  1.00it/s]

Successfully uploaded system


## 步骤 4 - 创建分析切片并进一步分析

你一共需要利用 Zeno 提供的接口创建 5 个不同的切片。

你可以首先创建两个切片：

1. 有标签的推文（含有"#"）
2. 有强烈正向情感的词语（如 love），你可以自行决定一个词

创建切片可以直接通过点击 "+" 按钮来进行，可以通过基本值匹配或正则表达式的方式创建，具体用法参见[文档](https://zenoml.com/docs/intro/)

![image.png](images/image.png)

有关 Zeno 中更多有趣的用法可以参考 Zeno 仓库中的 [README](https://github.com/zeno-ml/zeno)。

在此处写下你想要另外创建的三个切片，并在个人 Zeno 账户中创建，针对每一个创建的切片，用一两句话总结模型在其上的表现特点：


总体来说，roberta比GPT2表现好，关于前面两个切片，Hashtag似乎并没有很帮助到模型识别语气，模型在含"love"的数据上的准确率远高于平均；另值得注意的是，GPT2在标签为positive的贴文、及含强烈语气词汇时的表现与其他切片的相比差距很大。

3. 短推文：两个模型在此切片上准确率都高于平均，GPT2在此切片上表现接近roberta
4. 长推文：两个模型在此切片上准确率都低于平均，另外发现数据清洗不干净
5. 按照标签切片：两个模型对“neutral”语气的识别比对其他的两个语气差，另外GPT2对"positive"语气的识别表现远高于其他二者，GPT2对"negative"的识别是最差的


其他想法：
6. 含"IMAO"
7. 含emoji😂/🤣
不过数据集当中貌似不含这些元素

## 提交：
1. 本 notebook，保留你书写的代码与输出结果
2. 一张截图，显示你创建的 5 个切片